## [Dataset](https://www.kaggle.com/datasets/aljarah/xAPI-Edu-Data/data)

In [1]:
import pandas as pd

data = pd.read_csv("..\\data\\xAPI-Edu-Data.csv")
data

,gender,NationalITy,PlaceofBirth,StageID,GradeID,SectionID,Topic,Semester,Relation,raisedhands,VisITedResources,AnnouncementsView,Discussion,ParentAnsweringSurvey,ParentschoolSatisfaction,StudentAbsenceDays,Class
0,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,15,16,2,20,Yes,Good,Under-7,M
1,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,20,20,3,25,Yes,Good,Under-7,M
2,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,10,7,0,30,No,Bad,Above-7,L
3,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,30,25,5,35,No,Bad,Above-7,L
4,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,40,50,12,50,No,Bad,Above-7,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,F,Jordan,Jordan,MiddleSchool,G-08,A,Chemistry,S,Father,5,4,5,8,No,Bad,Above-7,L
476,F,Jordan,Jordan,MiddleSchool,G-08,A,Geology,F,Father,50,77,14,28,No,Bad,Under-7,M
477,F,Jordan,Jordan,MiddleSchool,G-08,A,Geology,S,Father,55,74,25,29,No,Bad,Under-7,M
478,F,Jordan,Jordan,MiddleSchool,G-08,A,History,F,Father,30,17,14,57,No,Bad,Above-7,L


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Завантаження даних
data = pd.read_csv("..\\data\\xAPI-Edu-Data.csv")

# Замінимо категоріальні дані на числові
label_encoder = LabelEncoder()

# Категоріальні змінні, де порядок має значення
data['GradeID'] = label_encoder.fit_transform(data['GradeID'])
data['StageID'] = label_encoder.fit_transform(data['StageID'])
data['SectionID'] = label_encoder.fit_transform(data['SectionID'])
data['Semester'] = label_encoder.fit_transform(data['Semester'])

# Для змінних, де порядок не має значення, використаємо OneHotEncoder
data = pd.get_dummies(data, columns=['gender', 'NationalITy', 'PlaceofBirth', 'Topic', 'Relation',
                                      'ParentAnsweringSurvey', 'ParentschoolSatisfaction', 'StudentAbsenceDays'])

# Вибір цільової змінної (наприклад, "Class") та ознак
X = data.drop('Class', axis=1)
y = data['Class']

# Масштабування числових ознак
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Розділяємо дані на тренувальну та тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Параметри для GridSearchCV
param_grid = {
    'C': [0.1, 1, 10],                      # Регуляризація
    'gamma': [0.01, 0.1, 1],                # Параметр для RBF ядра
    'kernel': ['linear', 'poly', 'rbf']     # Вибір ядра
}

# Використовуємо GridSearchCV для пошуку оптимальних параметрів
grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Найкращі параметри
best_params = grid_search.best_params_
print(f"Найкращі параметри: {best_params}")

# Навчання моделі з найкращими параметрами
best_svm = grid_search.best_estimator_

# Прогнозування на тестовій вибірці
y_pred = best_svm.predict(X_test)

# Оцінка точності
accuracy = accuracy_score(y_test, y_pred)
print(f"Точність моделі: {accuracy}")

# Точність, повнота, точність та F1-оцінка
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Точність: {precision}")
print(f"Повнота: {recall}")
print(f"F1-оцінка: {f1}")


Найкращі параметри: {'C': 0.1, 'gamma': 0.01, 'kernel': 'linear'}
Точність моделі: 0.7708333333333334
Точність: 0.7703659188034189
Повнота: 0.7708333333333334
F1-оцінка: 0.7701528687309671
